In [1]:
%load_ext autotime
import pandas as pd
import os
import geopandas as gpd
from glob import glob
from shapely.validation import make_valid
from tqdm.auto import tqdm
import time
pd.set_option("display.max_colwidth", 100)

In [36]:
files = pd.DataFrame({"filepath": glob("Asset Directory/**", recursive=True)})
files = files[files["filepath"].str.endswith((".shp", ".zip", ".gpkg", ".geojson"))]
files["filename"] = files["filepath"].apply(lambda x: os.path.splitext(os.path.basename(x))[0])
files["filesize_GB"] = files["filepath"].apply(lambda x: os.path.getsize(x) / 1e9)
files.sort_values("filesize_GB", ascending=False).reset_index(drop=True)

,filepath,filename,filesize_GB
0,Asset Directory/LUCAS NZ Land Use/lucas-nz-land-use-map-1990-2008-2012-2016-v011.shp,lucas-nz-land-use-map-1990-2008-2012-2016-v011,1.435426
1,Asset Directory/Koordinate/smap-soil-drainage-dec-2023/smap-soil-drainage-dec-2023.shp,smap-soil-drainage-dec-2023,0.699233
2,Asset Directory/lds-new-zealand-17layers-SHP/nz-building-outlines/nz-building-outlines.shp,nz-building-outlines,0.636510
3,Asset Directory/lris-fsl-soil-drainage-class-SHP/fsl-soil-drainage-class.shp,fsl-soil-drainage-class,0.164699
4,Asset Directory/Koordinate/nzlri-land-use-capability-2021/nzlri-land-use-capability-2021.shp,nzlri-land-use-capability-2021,0.164578
5,Asset Directory/DOC/DOC_Public_Conservation_Land.shp,DOC_Public_Conservation_Land,0.139802
6,Asset Directory/Koordinate/protected-areas/protected-areas.shp,protected-areas,0.134675
7,Asset Directory/NationalRoadCentrelineDataset/roads.shp,roads,0.130881
8,Asset Directory/lds-new-zealand-17layers-SHP/nz-road-centrelines-topo-150k.shp,nz-road-centrelines-topo-150k,0.116422
9,Asset Directory/lds-new-zealand-17layers-SHP/nz-track-centrelines-topo-150k/nz-track-centrelines...,nz-track-centrelines-topo-150k,0.074558


In [16]:
polys = []
for f in tqdm(glob("Projections/*_polygon.shp")):
  df = gpd.read_file(f)
  df.geometry = df.geometry.apply(make_valid)
  polys.append(df.union_all())

  0%|          | 0/6 [00:00<?, ?it/s]

In [17]:
for row in tqdm(files.itertuples(index=False), total=len(files)):
  print(f"Now processing {row.filename}")
  start = time.time()
  asset = gpd.read_file(row.filepath)#, engine="pyogrio", use_arrow=True)
  print(f"Time to read file: {time.time() - start}")
  start = time.time()
  clipped_asset = pd.concat(gpd.clip(asset, poly) for poly in tqdm(polys))
  print(f"Time to clip: {time.time() - start}")
  clipped_asset["affected_area"] = clipped_asset.area
  clipped_asset["affected_length"] = clipped_asset.length
  clipped_asset.to_file("affected_assets.gpkg", layer=row.filename, driver='GPKG')

  0%|          | 0/102 [00:00<?, ?it/s]

Now processing lucas-nz-land-use-map-1990-2008-2012-2016-v011
Time to read file: 20.920231580734253


  0%|          | 0/6 [00:00<?, ?it/s]

Time to clip: 87.12715315818787
Now processing fsl-soil-drainage-class
Time to read file: 2.0719239711761475


  0%|          | 0/6 [00:00<?, ?it/s]

...

Time to clip: 3.022231340408325
Now processing smap-soil-drainage-dec-2023
Time to read file: 15.872958660125732


  0%|          | 0/6 [00:00<?, ?it/s]

Time to clip: 20.277727603912354
Now processing nzlri-land-use-capability-2021
Time to read file: 4.642675876617432


  0%|          | 0/6 [00:00<?, ?it/s]

Time to clip: 15.689075469970703


In [18]:
gpd.list_layers("affected_assets.gpkg")

,name,geometry_type
0,location-and-extent-of-nzs-aquifers-2015,MultiPolygon
1,nz-primary-road-parcels,MultiPolygon
2,nz-roads-road-section-geometry,MultiLineString
3,nz-coastlines-topo-150k,MultiLineString
4,nz-orchard-polygons-topo-150k,MultiPolygon
5,nz-cadastral-survey-network-marks,Point
6,nz-trig-points-topo-150k,Point
7,nz-vineyard-polygons-topo-150k,Unknown
8,new-zealand-2022-estimated-resident-population-grid-250-metr,MultiPolygon
9,nz-swamp-polygons-topo-150k,MultiPolygon


In [23]:
files = files[~files.filename.isin(gpd.list_layers("affected_assets.gpkg").name)]
files

,filepath,filename,filesize_GB
87,Asset Directory/CAA NZ_Aerodromes.geojson,CAA NZ_Aerodromes,0.000152


In [26]:
for row in tqdm(files.itertuples(index=False), total=len(files)):
  print(f"Now processing {row.filename}")
  start = time.time()
  asset = gpd.read_file(row.filepath).to_crs(2193)#, engine="pyogrio", use_arrow=True)
  print(f"Time to read file: {time.time() - start}")
  start = time.time()
  clipped_asset = pd.concat(gpd.clip(asset, poly) for poly in tqdm(polys))
  print(f"Time to clip: {time.time() - start}")
  clipped_asset["affected_area"] = clipped_asset.area
  clipped_asset["affected_length"] = clipped_asset.length
  clipped_asset.to_file("affected_assets.gpkg", layer=row.filename, driver='GPKG')

  0%|          | 0/1 [00:00<?, ?it/s]

Now processing CAA NZ_Aerodromes
Time to read file: 0.030376195907592773


  0%|          | 0/6 [00:00<?, ?it/s]

Time to clip: 0.045610904693603516
